# Data Preprocessing

In [21]:
import numpy as np
import os
import pickle
import pandas as pd
import time
import warnings
import cv2
warnings.filterwarnings("ignore")
from numpy.random import permutation
np.random.seed(2016)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPool2D # GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import utils
from keras.models import model_from_json
from sklearn.metrics import log_loss, confusion_matrix
from keras import regularizers
import h5py

from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
# from keras.layers.advanced_activations import LeakyReLU
from sklearn.metrics import confusion_matrix

from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, \
    GlobalAveragePooling2D, Dense, BatchNormalization, Activation, Conv2D
from keras.models import Model
# from keras.engine.topology import get_source_inputs
from keras.losses import CategoricalCrossentropy
from keras.utils import get_source_inputs, plot_model
# from depthwise_conv2d import DepthwiseConvolution2D

In [2]:
use_cache = 1
dataset_path = r'../Project/Dataset/'
np.random.seed(2016)
warnings.filterwarnings("ignore")

In [3]:
def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(src=img, dsize=(224, 224), interpolation=cv2.INTER_LINEAR)
    return resized

In [4]:
def load_train():
    df = pd.read_csv(dataset_path + 'auc.distracted.driver.train.csv')
    x = df.iloc[:,0]
    y = df.iloc[:,1]
    X_train = []
    Y_train = []
    print('Read train images')
    for i in range (0,len(x)):
        fl= dataset_path + r'v1_cam1_no_split/' + r'/'.join(x[i].split('/')[-2:])
        print(f"Reading image {i}/{len(x)}")
        img = get_im_cv2(fl)
        X_train.append(img)
        Y_train.append(y[i])
    return X_train, Y_train

In [5]:
def load_valid():
    df = pd.read_csv(dataset_path + 'auc.distracted.driver.test.csv')
    x = df.iloc[:,0]
    y = df.iloc[:,1]
    X_valid = []
    Y_valid = []
    print('Read test images')
    for i in range (0,len(x)):
        fl = dataset_path + r'v1_cam1_no_split/' + r'/'.join(x[i].split('/')[-2:])
        print(f"Reading image {i}/{len(x)}")
        img = get_im_cv2(fl)
        X_valid.append(img)
        Y_valid.append(y[i])
    return X_valid, Y_valid

In [6]:
def cache_data(data, path):
    if os.path.isdir(os.path.dirname(path)):
        file = open(path, 'wb')
        pickle.dump(data, file)
        file.close()
    else:
        print('Directory doesnt exists')

In [7]:
def restore_data(path):
    data = dict()
    if os.path.isfile(path):
        file = open(path, 'rb')
        data = pickle.load(file)
    return data

In [8]:
def read_and_normalize_train_data():
    # cache_path = os.path.join('/cache', 'train_r_' + str(224) + '_c_' + str(224) + '_t_' + str(3) + '.dat')
    cache_path = r'../Project/Dataset/cache/train_v1.dat'
    if not os.path.isfile(cache_path) or use_cache == 0:
        train_data, train_target= load_train()
        cache_data((train_data, train_target), cache_path)
    else:
        print('Restore train from cache!')
        (train_data, train_target) = restore_data(cache_path)
    
    print('Convert to numpy...')
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)
    
    print('Reshape...')
    train_data = train_data.transpose((0, 1, 2, 3))

    # Normalise the train data
    print('Convert to float...')
    train_data = train_data.astype('float16')
    mean_pixel = [80.857, 81.106, 82.928]
    
    print('Substract 0...')
    train_data[:, :, :, 0] -= mean_pixel[0]
    
    print('Substract 1...')
    train_data[:, :, :, 1] -= mean_pixel[1]

    print('Substract 2...')
    train_data[:, :, :, 2] -= mean_pixel[2]

    train_target = utils.to_categorical(train_target, 10)
    
    # Shuffle experiment START !!
    perm = permutation(len(train_target))
    train_data = train_data[perm]
    train_target = train_target[perm]
    # Shuffle experiment END !!
    
    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target

In [9]:
def read_and_normalize_test_data():
    start_time = time.time()
    # cache_path = os.path.join('/cache', 'test_r_' + str(224) + '_c_' + str(224) + '_t_' + str(3) + '.dat')
    cache_path = r'../Project/Dataset/cache/test_v1.dat'

    if not os.path.isfile(cache_path) or use_cache == 0:
        test_data, test_target = load_valid()
        cache_data((test_data, test_target ), cache_path)
    else:
        print('Restore test from cache [{}]!')
        (test_data, test_target) = restore_data(cache_path)

    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.transpose((0, 1, 2, 3))

    # Normalise the test data data

    test_data = test_data.astype('float16')
    mean_pixel = [80.857, 81.106, 82.928]

    test_data[:, :, :, 0] -= mean_pixel[0]

    test_data[:, :, :, 1] -= mean_pixel[1]

    test_data[:, :, :, 2] -= mean_pixel[2]

    test_target = utils.to_categorical(test_target, 10)
    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    print('Read and process test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return test_data, test_target

In [10]:
X_train, Y_train = read_and_normalize_train_data()
X_valid, Y_valid = read_and_normalize_test_data()

Restore train from cache!


Convert to numpy...
Reshape...
Convert to float...
Substract 0...
Substract 1...
Substract 2...
Train shape: (12977, 224, 224, 3)
12977 train samples
Restore test from cache [{}]!
Test shape: (4331, 224, 224, 3)
4331 test samples
Read and process test data time: 3.49 seconds


# Run Model

In [11]:
batch_size = 64
nb_epoch = 100

In [19]:
def VGG_16(input_tensor=None, input_shape=None, alpha=1, shallow=False, classes=10):

    if input_tensor is None:
        img_input = Input(shape =(224,224,3))
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    print(img_input)

    # 1st Conv Block

    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(img_input)
    x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    
    # 2nd Conv Block

    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    
    # 3rd Conv block

    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.1)(x)
    
    # 4th Conv block

    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x = Dropout(0.2)(x)

    # 5th Conv block

    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
    x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
    x= Dropout(0.3)(x)

    # th Conv block

    x = Conv2D (filters =512, kernel_size =7, padding ='same', activation='relu')(x)
    x= Dropout(0.4)(x)
    x = Conv2D (filters =512, kernel_size =1, padding ='same', activation='relu')(x)
    x= Dropout(0.5)(x)
    x = Conv2D (filters =10, kernel_size =1, padding ='same', activation='softmax')(x)
    # output = Dense(10, activation='softmax')(x)
    output = GlobalAveragePooling2D()(x)
    
    # creating the model

    model = Model (inputs=img_input, outputs =output)
    model.summary()    

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, output, name='VGG_16')
    # model.load_weights('weights.h5')
    adam = SGD(learning_rate=0.0001, momentum=0.9, weight_decay=10e-6)

    model.compile(adam, loss=CategoricalCrossentropy(),metrics=['accuracy'])
    return model

In [25]:
print(Y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [22]:
model = VGG_16()

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_32 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 conv2d_33 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 112, 112, 64)      0         
 ng2D)                                                           
                                                                 
 conv2d_34 (Conv2D)          (None, 112, 112, 128)     73856     
                                      

In [14]:
# plot_model(model, show_shapes=True, show_layer_names=True)

In [15]:
#Data augmentation
datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            shear_range=0.2
            )

datagen.fit(X_train)

In [23]:
weights_path=r'./Checkpoint/'       
callbacks = [ModelCheckpoint(weights_path, monitor='val_acc', save_best_only=True, verbose=1)]

print(X_train.shape)
hist=model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size), epochs=nb_epoch, verbose=1, validation_data=(X_valid, Y_valid), callbacks=callbacks)

(12977, 224, 224, 3)
Epoch 1/100


2024-03-07 20:11:43.774242: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inVGG_16/dropout_10/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-07 20:11:44.095690: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-03-07 20:11:44.095730: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 6348288093820418842


UnknownError: Graph execution error:

Detected at node categorical_crossentropy/Log defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ubuntu/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ubuntu/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ubuntu/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ubuntu/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/ubuntu/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 

In [ ]:
pd.DataFrame(hist.history).to_csv(r'../Project/Dataset/cache/try_hist.csv')

predictions_valid = model.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=1)
cm1=confusion_matrix(Y_valid.argmax(axis=1), predictions_valid.argmax(axis=1))
ss=cm1[0,0]+cm1[1,1]+cm1[2,2]+cm1[3,3]+cm1[4,4]+cm1[5,5]+cm1[6,6]+cm1[7,7]+cm1[8,8]+cm1[9,9];
test_accuracy=np.divide(ss,4331);
print('Test Accuracy:',test_accuracy)

ppath=os.path.join('/home/gpu3/Desktop/mobileVGG','cache','confusion_mat.npy')
np.save(ppath, cm1)